# 방법 1

In [1]:
from google.cloud import bigquery
import os

# 서비스 계정 키 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Admin/Desktop/de_basic/gcp_tutorial/lgu6h-project-09e19171e4dd.json"

# BigQuery 클라이언트 초기화
client = bigquery.Client()

# BigQuery 테이블 참조 정보 정의
project_id = "lgu6h-project"
dataset_id = "sample_data"
table_id = "creditcard_fraud"
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# 로컬 CSV 파일 경로 지정
file_path = "C:/Users/Admin/Desktop/de_basic/gcp_tutorial/creditcard.csv"

# BigQuery 테이블 스키마 정의
# Time 컬럼: FLOAT64 타입
# V1~V28 컬럼: FLOAT64 타입 (특성 변수들)
# Amount 컬럼: FLOAT64 타입 (거래 금액)
# Class 컬럼: INTEGER 타입 (사기 여부: 0=정상, 1=사기)
schema = [bigquery.SchemaField("Time", "FLOAT64")] + \
         [bigquery.SchemaField(f"V{i}", "FLOAT64") for i in range(1, 29)] + [
             bigquery.SchemaField("Amount", "FLOAT64"),
             bigquery.SchemaField("Class", "INTEGER"),
         ]

# BigQuery 로드 작업 설정
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,  # 소스 파일 형식: CSV
    skip_leading_rows=1,                      # 헤더 행 건너뛰기
    schema=schema,                            # 미리 정의된 스키마 사용
    write_disposition="WRITE_TRUNCATE"        # 기존 테이블 덮어쓰기
)

# CSV 파일을 BigQuery로 업로드
with open(file_path, "rb") as source_file:
    load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

# 업로드 진행 상황 출력
print("업로드 중...")
load_job.result()  # 업로드 완료까지 대기
print("업로드 완료!")

# 업로드 결과 확인 및 출력
table = client.get_table(table_ref)
print(f"{table.num_rows} rows loaded to {table.project}.{table.dataset_id}.{table.table_id}")

업로드 중...
업로드 완료!
284807 rows loaded to lgu6h-project.sample_data.creditcard_fraud


# 방법 2

In [2]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
from google.cloud import bigquery  # BigQuery 클라이언트 라이브러리
import os  # 운영체제 관련 기능 (환경변수 설정 등)

# Google Cloud 서비스 계정 인증 설정
# 서비스 계정 키 파일 경로를 환경변수로 설정하여 BigQuery 접근 권한 획득
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Admin/Desktop/de_basic/gcp_tutorial/lgu6h-project-09e19171e4dd.json"

# 1단계: Pandas를 사용하여 로컬 CSV 파일을 DataFrame으로 로드
# read_csv() 함수로 CSV 파일을 읽어와서 메모리에 DataFrame 형태로 저장
df = pd.read_csv("C:/Users/Admin/Desktop/de_basic/gcp_tutorial/creditcard.csv")

# 2단계: BigQuery 클라이언트 객체 생성
# 이 클라이언트를 통해 BigQuery 서비스와 통신하여 데이터 업로드/다운로드 수행
client = bigquery.Client()

# 3단계: BigQuery 테이블 정보 정의
project_id = "lgu6h-project"      # Google Cloud 프로젝트 ID
dataset_id = "sample_data"        # BigQuery 데이터셋 ID
table_id = "creditcard_fraud"     # BigQuery 테이블 ID
# 전체 테이블 식별자 생성 (project.dataset.table 형식)
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# 4단계: BigQuery 업로드 작업 설정
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # 기존 테이블이 있으면 완전히 덮어쓰기
    autodetect=True                      # 스키마 자동 감지 (컬럼 타입 자동 추론)
)

# 5단계: Pandas DataFrame을 BigQuery 테이블로 업로드
# load_table_from_dataframe() 메서드로 DataFrame을 BigQuery에 전송
job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
job.result()  # 업로드 작업이 완료될 때까지 대기 (동기 실행)

# 6단계: 업로드 완료 결과 출력
# DataFrame의 행 개수와 업로드된 테이블 정보를 출력하여 성공 여부 확인
print(f"{df.shape[0]} rows uploaded to {full_table_id}")


284807 rows uploaded to lgu6h-project.sample_data.creditcard_fraud


# 방법 3

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
import os  # 운영체제 관련 기능 (환경변수 설정 등)

# Google Cloud 서비스 계정 인증 설정
# 서비스 계정 키 파일 경로를 환경변수로 설정하여 BigQuery 접근 권한 획득
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Admin/Desktop/de_basic/gcp_tutorial/lgu6h-project-09e19171e4dd.json"

# 1단계: Pandas를 사용하여 로컬 CSV 파일을 DataFrame으로 로드
# read_csv() 함수로 CSV 파일을 읽어와서 메모리에 DataFrame 형태로 저장
df = pd.read_csv("C:/Users/Admin/Desktop/de_basic/gcp_tutorial/creditcard.csv")

# 2단계: BigQuery 테이블 정보 정의
table_id = "sample_data.creditcard_fraud"  # 데이터셋.테이블명 형식 (프로젝트는 별도 지정)
project_id = "lgu6h-project"               # Google Cloud 프로젝트 ID

# 3단계: Pandas DataFrame을 BigQuery 테이블로 업로드
# to_gbq() 메서드를 사용하여 DataFrame을 BigQuery에 직접 전송
# if_exists="replace": 기존 테이블이 있으면 완전히 덮어쓰기
df.to_gbq(
    destination_table=table_id,  # 대상 테이블 식별자
    project_id=project_id,       # 프로젝트 ID
    if_exists="replace"          # 기존 테이블 처리 방식 (덮어쓰기)
)

print("업로드 완료!")

# 4단계: BigQuery에서 데이터를 다시 가져오기 (검증 목적)
# read_gbq() 메서드를 사용하여 SQL 쿼리로 데이터 조회
df_from_bq = pd.read_gbq(
    query="SELECT * FROM `lgu6h-project.sample_data.creditcard_fraud`",  # 전체 데이터 조회 쿼리
    project_id=project_id,  # 프로젝트 ID
    dialect="standard"      # BigQuery SQL 방언 사용
)

# 5단계: 업로드 결과 검증
# DataFrame의 행 개수를 출력하여 성공적으로 데이터가 업로드되었는지 확인
print(f"불러온 행 개수: {len(df_from_bq)}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_23740\3171931068.py:20: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(


업로드 완료!


C:\Users\Admin\AppData\Local\Temp\ipykernel_23740\3171931068.py:30: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_from_bq = pd.read_gbq(


불러온 행 개수: 284807


# 쿼리 통해서 데이터 가져오기

In [4]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
from google.cloud import bigquery  # Google BigQuery 클라이언트 라이브러리
import os  # 운영체제 관련 기능 (환경변수 설정 등)
import time  # 시간 측정을 위한 라이브러리
from datetime import timedelta  # 시간 간격 계산을 위한 라이브러리

# 시간 포맷 함수 정의
# 초 단위 시간을 시:분:초 형태의 문자열로 변환하는 함수
def format_time(seconds):
    return str(timedelta(seconds=int(seconds)))

# 전체 프로세스 시작 시간 기록
# 성능 측정을 위해 전체 실행 시간을 추적하기 위한 시작점 설정
total_start = time.time()

# 0. Google Cloud 서비스 계정 인증 설정
# 서비스 계정 키 파일 경로를 환경변수로 설정하여 BigQuery 접근 권한 획득
# 이 파일에는 Google Cloud 프로젝트에 대한 인증 정보가 포함되어 있음
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Admin/Desktop/de_basic/gcp_tutorial/lgu6h-project-09e19171e4dd.json"

# 1. 로컬 CSV 파일을 DataFrame으로 로드
# read_csv() 함수를 사용하여 CSV 파일을 메모리에 DataFrame 형태로 읽어옴
start = time.time()  # CSV 로드 시작 시간 기록
df = pd.read_csv("C:/Users/Admin/Desktop/de_basic/gcp_tutorial/creditcard.csv")
print(f"✅ CSV 로드 완료: {df.shape[0]} rows (소요: {format_time(time.time() - start)})")

# 2. BigQuery 클라이언트 객체 생성
# Google Cloud BigQuery 서비스와 통신하기 위한 클라이언트 인스턴스 생성
# 이 클라이언트를 통해 데이터 업로드, 쿼리 실행 등의 작업 수행
client = bigquery.Client()

# 3. BigQuery 테이블 정보 정의
# 프로젝트, 데이터셋, 테이블 ID를 개별적으로 정의하고 전체 테이블 식별자 생성
project_id = "lgu6h-project"    # Google Cloud 프로젝트 ID
dataset_id = "sample_data"      # BigQuery 데이터셋 ID
table_id = "creditcard_fraud"   # BigQuery 테이블 ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"  # 전체 테이블 식별자 (프로젝트.데이터셋.테이블)

# 4. BigQuery 업로드 작업 설정
# LoadJobConfig 객체를 생성하여 데이터 업로드 시의 동작 방식 정의
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # 기존 테이블이 있으면 완전히 덮어쓰기
    autodetect=True                      # 스키마 자동 감지 (컬럼 타입 자동 설정)
)

# 5. DataFrame을 BigQuery 테이블로 업로드
# load_table_from_dataframe() 메서드를 사용하여 Pandas DataFrame을 BigQuery에 전송
start = time.time()  # 업로드 시작 시간 기록
print("📤 BigQuery 업로드 중...")
job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)  # 비동기 업로드 작업 시작
job.result()  # 업로드 작업이 완료될 때까지 대기 (동기화)
print(f"✅ 업로드 완료 (소요: {format_time(time.time() - start)})")

# 6. BigQuery에서 데이터 조회 (검증 목적)
# SQL 쿼리를 실행하여 업로드된 데이터를 다시 가져와서 검증
start = time.time()  # 쿼리 시작 시간 기록
query = f"SELECT * FROM `{full_table_id}` LIMIT 10"  # 전체 데이터에서 상위 10행만 조회하는 SQL 쿼리
df_from_bq = client.query(query).to_dataframe()  # 쿼리 실행 결과를 DataFrame으로 변환
print(f"🔍 쿼리 완료 (소요: {format_time(time.time() - start)})")

# 7. 결과 데이터 출력 및 검증
# 업로드된 데이터의 일부를 출력하여 성공적으로 처리되었는지 확인
print("상위 5행:")
print(df_from_bq.head())  # DataFrame의 처음 5행 출력
print(f"📊 BigQuery에서 불러온 총 행 수: {len(df_from_bq)}")  # 조회된 총 행 수 출력

# 전체 프로세스 소요 시간 출력
# CSV 로드부터 BigQuery 쿼리까지 전체 과정에 걸린 시간을 계산하여 출력
print(f"⏱️ 전체 소요 시간: {format_time(time.time() - total_start)}")

✅ CSV 로드 완료: 284807 rows (소요: 0:00:00)
📤 BigQuery 업로드 중...
✅ 업로드 완료 (소요: 0:00:08)
🔍 쿼리 완료 (소요: 0:00:00)
상위 5행:
    Time        V1        V2        V3        V4        V5        V6  \
0  282.0 -0.356466  0.725418  1.971749  0.831343  0.369681 -0.107776   
1  380.0 -1.299837  0.881817  1.452842 -1.293698 -0.025105 -1.170103   
2  403.0  1.237413  0.512365  0.687746  1.693872 -0.236323 -0.650232   
3  430.0 -1.860258 -0.629859  0.966570  0.844632  0.759983 -1.481173   
4  711.0 -0.431349  1.027694  2.670816  2.084787 -0.274567  0.286856   

         V7        V8        V9  ...       V21       V22       V23       V24  \
0  0.751610 -0.120166 -0.420675  ...  0.020804  0.424312 -0.015989  0.466754   
1  0.861610 -0.193934  0.592001  ... -0.272563 -0.360853  0.223911  0.598930   
2  0.118066 -0.230545 -0.808523  ... -0.077543 -0.178220  0.038722  0.471218   
3 -0.509681  0.540722 -0.733623  ...  0.268028  0.125515 -0.225029  0.586664   
4  0.152110  0.200872 -0.596505  ...  0.001241  0.154170

c:\Users\Admin\Desktop\de_basic\gcp_tutorial\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
